In [19]:
import sqlite3
import csv

conn = sqlite3.connect('airbnb.db')
c = conn.cursor()

In [20]:
def create_database(conn):
    """
    Using the database connection conn, create tables appropriate for 
    storing and using IMDB data.  Drops any pre-exsiting tables that would be re-defined by this procedure.
    """
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS 'listings' ''')
    c.execute('''DROP TABLE IF EXISTS 'neighborhoods' ''')
    c.execute('''DROP TABLE IF EXISTS 'calendar' ''')
    c.execute('''DROP TABLE IF EXISTS 'face_table' ''')
    c.execute('''CREATE TABLE listings(
                    id INTEGER NOT NULL PRIMARY KEY,
                    listing_url VARCHAR(100) NOT NULL DEFAULT '',
                    name VARCHAR(100) NOT NULL DEFAULT '',
                    summary VARCHAR(100) NOT NULL DEFAULT '',
                    space VARCHAR(100) NOT NULL DEFAULT '',
                    description VARCHAR(100) NOT NULL DEFAULT '',
                    neighborhood_overview VARCHAR(100) NOT NULL DEFAULT '',
                    picture_url VARCHAR(100) NOT NULL DEFAULT '',
                    street VARCHAR(100) NOT NULL DEFAULT '',
                    neighbourhood_cleansed VARCHAR(100) NOT NULL DEFAULT '',
                    city VARCHAR(100) NOT NULL DEFAULT '',
                    latitude REAL NOT NULL,
                    longitude REAL NOT NULL,
                    property_type VARCHAR(100) NOT NULL DEFAULT '',
                    room_type VARCHAR(100) NOT NULL DEFAULT '',
                    accommodates INT NOT NULL,
                    bathrooms REAL NOT NULL,
                    bedrooms INT NOT NULL,
                    beds INT NOT NULL,
                    price REAL NOT NULL,
                    instant_bookable VARCHAR(100) NOT NULL DEFAULT '',
                    cancellation_policy VARCHAR(100) NOT NULL DEFAULT '',
                    sum_of_nights INT NOT NULL
                    )''')
    c.execute('''CREATE TABLE neighborhoods(
                    id INTEGER NOT NULL PRIMARY KEY,
                    neighborhood VARCHAR(100) NOT NULL DEFAULT ''
                    )''')
    # month_year, mean_price, avaliable need to fill or not
    c.execute('''CREATE TABLE calendar(
                    id INTEGER NOT NULL PRIMARY KEY,
                    listing_id INTEGER NOT NULL,
                    month_year INTEGER NOT NULL,
                    mean_price INTEGER NOT NULL,
                    FOREIGN KEY(listing_id) REFERENCES listings(id)
                    )''')
    c.execute('''CREATE TABLE face_table(
                    neighborhood_id INTEGER NOT NULL,
                    listing_id INTEGER NOT NULL, 
                    month_id INTEGER NOT NULL,
                    FOREIGN KEY(neighborhood_id) REFERENCES neighborhoods(id),
                    FOREIGN KEY(listing_id) REFERENCES listings(id),
                    FOREIGN KEY(month_id) REFERENCES calendar(id)
                    )''')
    conn.commit()
create_database(conn)

0 id

1 listing_url

2 name

3 summary

4 space

5 description

6 neighborhood_overview

7 picture_url

8 street

9 neighbourhood_cleansed

10 city

11 latitude

12 longitude

13 property_type

14 room_type

15 accommodates

16 bathrooms

17 bedrooms

18 beds

19 price

20 instant_bookable

21 cancellation_policy

22 sum_of_nights

In [21]:
def populate_database(conn, csv_file):    
    reader = csv.reader(open(csv_file))

    listing_dict = dict()
    calendar_dict = dict()
    neighborhood_dict = dict()
    
    month_id = 0
    neighborhood_id = 0
    next(reader, None)
    for row in reader:
        row = [item.lower() for item in row]
        
        # insert into dimension tables
        try:
            c.execute('''INSERT INTO listings 
            (id,listing_url,name,summary,space,description,
            neighborhood_overview,picture_url,street,neighbourhood_cleansed,
            city,latitude,longitude,property_type,room_type,accommodates,
            bathrooms,bedrooms,beds,price,instant_bookable,cancellation_policy,sum_of_nights)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
                      (row[0], row[1], row[2],row[3] ,row[4], row[5], row[6], row[7], row[8], row[9],
                      row[10], row[11], row[12], row[13], row[14], row[15], row[16], row[17],
                      row[18], row[19], row[20], row[21], row[22]))
        except sqlite3.Error as e:
            print("listings, insert error:", e.args[0])
        
        
        if row[9] not in neighborhood_dict.keys():
            neighborhood_id += 1
            neighborhood_dict[row[9]] = neighborhood_id
            try:
                c.execute('''INSERT INTO neighborhoods 
                (id,neighborhood) VALUES (?, ?)''',
                         (neighborhood_dict[row[9]], row[9]))
            except sqlite3.Error as e:
                print("neighborhoods, insert error:", e.args[0])
        
        if row[23] not in calendar_dict.keys():
            month_id += 1
            calendar_dict[row[23]] = month_id
            try:
                c.execute('''INSERT INTO calendar 
                (id,listing_id,month_year, mean_price) VALUES (?, ?, ?, ?)''',
                         (calendar_dict[row[23]], row[0], row[23], row[24]))
            except sqlite3.Error as e:
                print("calendar, insert error:", e.args[0])
       
         # insert fact table
        try:
            c.execute('''
                INSERT INTO fact_table (listing_id,neighborhood_id,month_id) 
                SELECT listings.id, neighborhoods.id,calendar.id 
                FROM listings, calendar, neighborhoods
                WHERE listings.name = ? AND neighborhoods.neighborhood = ? AND calendars.month_year = ?
                ''', (row[1], row[9], row[23]))
        except sqlite3.Error as e:
            print("fact_table, insert error:", e.args[0])
        conn.commit()
        
populate_database(conn, 'data/cleansed_listings.csv')

In [22]:
%reload_ext sql
%sql sqlite:///airbnb.db

'Connected: None@airbnb.db'

In [23]:
%%sql
SELECT count(name) FROM listings

Done.


count(name)
4866


In [26]:
%%sql
SELECT * FROM neighborhoods ORDER BY neighborhood

Done.


id,neighborhood
25,allston
11,back bay
5,bay village
15,beacon hill
18,brighton
13,charlestown
7,chinatown
22,dorchester
16,downtown
12,east boston
